# Import

In [ ]:
!pip install scrapetube
!pip install yt_dlp
!pip install selenium
!pip install rutube-downloader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 312.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 6.6 MB/s eta 0:00:00
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210082 sha256=3b282715b9f7a446566ecf58b35c7b56a074fa750422127fdac520cf82735428
  Stored in directory: /root/.cache/pip/wheels/ee/3b/0b/1b865800e916d671a24028d884698674138632a83fdfad4926
Successfully built grapheme


In [ ]:
# base
import pandas as pd
from tqdm import tqdm
import os
import time
tqdm.pandas()
from concurrent.futures import ProcessPoolExecutor

# parser
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# youtube
import scrapetube
import yt_dlp

# rutube
from rutube import Rutube

# visualization
import plotly.express as px
import plotly.graph_objs as go

# cv2
import cv2

In [ ]:
# директория со всеми промежуточными и итоговыми датасетами
DATASETS_PATH = r'/content/drive/MyDrive/диплом/Datasets'
# путь до хром драйвера
DRIVER_PATH = os.path.join('C:\\','chromedriver-win64_131','chromedriver.exe')
# путь, куда скачиваются полные видеозаписи выступления гимнасток
OUTPUT_PATH = r'/content/drive/MyDrive/диплом/YouTube_video'
# размеченные видео по классам элементов
ELEMENTS_PATH = r'/content/drive/MyDrive/диплом/По элементам'


# YouTube

Для анализа и сбора видео с платформы YouTube выбраны следущие каналы:
- rgymrussia: https://www.youtube.com/@RGYMRUSSIA/videos
- noname: https://www.youtube.com/@Noname-w2w/videos
- Top Tier RG: https://www.youtube.com/@flyerthantheg6

In [ ]:
# каналы youtube, из которых будут извлечены видеозаписи
channels = [
    'UCCyTxSkhfeMjnQBqDkY7H7A',   # rgymrussia
    'UCgOPElGlY_SLbdY8Yv8xEhg', # noname, https://www.youtube.com/@Noname-w2w/videos,
    'UCxkgNSdzHdyPmG0-fjgLWdw',  # top tier rg
]

## extract links

In [ ]:
# По ID канала на ютуб получаем ссылки и описание на все видеозаписи этого канала

df_youtube = pd.DataFrame(columns= ['channel_id','video_url','video_name','length_video','date_publication'])  # датасет с ссылками и описанием видеозаписей
for channel_id in channels:
  print(f'ID канала: {channel_id}')
  videos = scrapetube.get_channel(channel_id)  # получаем все ссылки на видео из канала

  df_channel = pd.DataFrame(columns = ['channel_id','video_url','video_name','length_video','date_publication'])
  for video in tqdm(videos):

      df_video = pd.DataFrame({'channel_id': channel_id,  # id канала
                          'video_url': f'https://www.youtube.com/watch?v={video["videoId"]}', # ссылка на видео
                          'video_name': video['title']['runs'][0]['text'] , # названия видео
                          'length_video' : video[ 'lengthText']['simpleText'], # длина видео
                          'date_publication': video['publishedTimeText']['simpleText']},index=[0])  # сколько времени прошло с момента публикации видео
      df_channel = pd.concat([df_channel, df_video], axis = 0)

  df_youtube = pd.concat([df_youtube, df_channel], axis = 0)
display(df_youtube.head(3))
df_youtube.shape

ID канала: UCCyTxSkhfeMjnQBqDkY7H7A


5054it [03:15, 25.87it/s]


ID канала: UCgOPElGlY_SLbdY8Yv8xEhg


509it [00:18, 27.59it/s]


ID канала: UCxkgNSdzHdyPmG0-fjgLWdw


763it [00:28, 26.80it/s]


,channel_id,video_url,video_name,length_video,date_publication
0,UCCyTxSkhfeMjnQBqDkY7H7A,https://www.youtube.com/watch?v=S_RiykbFdQQ,Kira Yablochnikova Ribbon AA Children of Asia ...,1:37,8 months ago
0,UCCyTxSkhfeMjnQBqDkY7H7A,https://www.youtube.com/watch?v=Tsq1rblKdcM,Arina Kovshova Ribbon AA Children of Asia 2024,1:37,8 months ago
0,UCCyTxSkhfeMjnQBqDkY7H7A,https://www.youtube.com/watch?v=CelSCMNJBGQ,Arina Kovshova Clubs AA Children of Asia 2024,1:41,8 months ago


(6326, 5)

In [ ]:
# макс видео может длиться 3 минуты
df_youtube['length_video'].apply(lambda x: len(x.split(':'))).max()

3

In [ ]:
# преобразование длительности видео в секунды
def convert_duration(x):
    if len(x.split(":")) == 2:
        return int(x.split(":")[0])*60 + int(x.split(":")[1])
    elif len(x.split(":")) == 3:
        return int(x.split(":")[0])*3600 + int(x.split(":")[1])*60 + int(x.split(":")[2])
    else:
        return 0
df_youtube['duration_video'] = df_youtube['length_video'].apply(convert_duration)
df_youtube[['length_video', 'duration_video']].head(3)

,length_video,duration_video
0,1:37,97
0,1:37,97
0,1:41,101


In [ ]:
# rgymrussia: https://www.youtube.com/@RGYMRUSSIA/videos
# noname: https://www.youtube.com/@Noname-w2w/videos
# Top Tier RG: https://www.youtube.com/@flyerthantheg6

# название канала
channel_mapper = {'UCCyTxSkhfeMjnQBqDkY7H7A': 'RGYMRUSSIA',
                  'UCgOPElGlY_SLbdY8Yv8xEhg': 'Noname',
                  'UCxkgNSdzHdyPmG0-fjgLWdw': 'Top Tier RG'
                  }
df_youtube['channel_name'] = df_youtube['channel_id'].map(channel_mapper)
df_youtube.drop(columns = ['length_video'], inplace = True)
df_youtube.reset_index(drop = True, inplace = True)

In [ ]:
df_youtube.to_excel(os.path.join(DATASETS_PATH, 'YouTube_links.xlsx'), index = False)

In [ ]:
df_youtube = pd.read_excel(os.path.join(DATASETS_PATH, 'YouTube_links.xlsx'))

# RuTube

### Сбор видео по каналам


Выбранные каналы для анализа с платформы RuTube:

- photosport:https://rutube.ru/channel/24477762/videos
- Звезды Сибири: https://rutube.ru/channel/24695426/videos/

In [ ]:
channels = [
    'https://rutube.ru/channel/24477762/videos',  # photosport
    'https://rutube.ru/channel/24695426/videos/',   # звезды сибири
]

По каждому из каналов извлекаем ссылки на видео данного канала путем парсинга ссылок на каналы:

> Подключаемся к странице канала на Rutube, динамически прокручиваем страницу до самого конца, извлекая все необходимые данные по каждому видео канала

In [ ]:
# подключение к странице канала на рутуб
def get_source(url):
    options = webdriver.ChromeOptions()
    service = Service(executable_path=DRIVER_PATH)
    driver = webdriver.Chrome(service=service, options=options)
    driver.maximize_window()


    try:
        driver.get(url=url)
        time.sleep(1)  # время на загрузку страницы
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:  # листаем страницу до последнего пикселя
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)  # ожидание загрузки страницы

#             новый скролл
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                # после того как драйвер долистал до конца страницы записываем всю разметку в отдельный файл
                with open("source_page.html", "w", encoding="utf-8") as file:
                    file.write(driver.page_source)
                break
            last_height = new_height

    except Exception as _ex:
        print(_ex)
    finally:  # закрываем страницу
        driver.close()
        driver.quit()

In [ ]:
# парсим информацию по видео из канала
def get_items(file_path):
    with open(file_path, encoding='utf-8') as file:
        src = file.read()  # считываем html файл

    soup = BeautifulSoup(src, 'lxml')
    items_divs = soup.find_all('article', class_='wdp-card-wrapper-module__wrapper')  # ищем все карты видео

    rut_url = 'https://rutube.ru'

    data = []

    for item in items_divs:
#         <a class="wdp-link-module__link wdp-card-description-module__author wdp-card-description-module__url" title="Интересная наука"
        item_channel = soup.select_one("a.wdp-link-module__link.wdp-card-description-module__author.wdp-card-description-module__url")['title']

        item_url = item.find('a', class_='wdp-link-module__link wdp-card-poster-module__posterWrapper').get('href')
        item_name = item.find('a', class_='wdp-link-module__link wdp-card-description-module__title '
                                           'wdp-card-description-module__url wdp-card-description-module__videoTitle')
        item_duration  = item.find("span", class_="wdp-poster-badge-module__poster-badge wdp-card-video-options-module__duration")

        data.append([item_channel, url, rut_url + item_url, item_name.text, item_duration.text])
    df = pd.DataFrame(data, columns = ['channel_name', 'channel_url', 'video_url', 'video_name','video_duration'])  #, 'description', 'duration'])

    return df

In [ ]:
df_rutube = pd.DataFrame()
for url in channels:
    get_source(url = url)
    channel_df = get_items(file_path=r'source_page.html')
    df_rutube = pd.concat([df_rutube, channel_df], axis = 0)
display(df_rutube.head(5))
df_rutube.reset_index(inplace=True, drop=True)
df_rutube.to_excel(os.path.join(DATASETS_PATH,'RuTube_links.xlsx'), index = False)

In [ ]:
df_rutube = pd.read_excel(os.path.join(DATASETS_PATH,'RuTube_links.xlsx'))
display(df_rutube.head(3))
df_rutube.shape

,channel_name,channel_url,video_url,video_name,video_duration
0,Photoprosport,https://rutube.ru/channel/24477762/videos,https://rutube.ru/video/550c3c15b39c67d2d14dd9...,2023-02-16 Римарачин Диас Николь ЛЕНТА (Первен...,01:32
1,Photoprosport,https://rutube.ru/channel/24477762/videos,https://rutube.ru/video/180c31d5722ca9a591dd78...,Молодёжная мужская сборная Красноярского края ...,02:44
2,Photoprosport,https://rutube.ru/channel/24477762/videos,https://rutube.ru/video/b72ebc3fb0cec1493bd4c7...,2023-02-16 Никольская Софья БУЛАВЫ (финал мног...,01:33


(317, 5)

In [ ]:
df_rutube['video_duration'].str.split(':').apply(lambda x: len(x)).unique()  # мин: сек

array([2])

In [ ]:
# преобразование длительности видео в секунды
df_rutube['video_duration'] = df_rutube['video_duration'].str.split(':').apply(lambda x: int(x[0])*60 + int(x[1]))
df_rutube.head(2)

,channel_name,channel_url,video_url,video_name,video_duration
0,Photoprosport,https://rutube.ru/channel/24477762/videos,https://rutube.ru/video/550c3c15b39c67d2d14dd9...,2023-02-16 Римарачин Диас Николь ЛЕНТА (Первен...,92
1,Photoprosport,https://rutube.ru/channel/24477762/videos,https://rutube.ru/video/180c31d5722ca9a591dd78...,Молодёжная мужская сборная Красноярского края ...,164


# Statistic videodataset

## concat youtube + rutube

In [ ]:
df_youtube.columns

Index(['channel_id', 'video_url', 'video_name', 'date_publication',
       'duration_video', 'channel_name'],
      dtype='object')

In [ ]:
df_rutube.columns

Index(['channel_name', 'channel_url', 'video_url', 'video_name',
       'video_duration'],
      dtype='object')

In [ ]:
# собираем 1 общий датасет с видеозаписями из ютуба и рутуба
df_youtube['platform'] = 'YouTube'
df_rutube['platform'] = 'RuTube'

df = pd.concat([
    df_youtube.rename(columns = {'duration_video':'video_duration'})[['platform','channel_name', 'video_url', 'video_duration']],
    df_rutube[['platform','channel_name', 'video_url', 'video_duration']]
])
display(df.head(4))
df.shape

,platform,channel_name,video_url,video_duration
0,YouTube,RGYMRUSSIA,https://www.youtube.com/watch?v=S_RiykbFdQQ,97
1,YouTube,RGYMRUSSIA,https://www.youtube.com/watch?v=Tsq1rblKdcM,97
2,YouTube,RGYMRUSSIA,https://www.youtube.com/watch?v=CelSCMNJBGQ,101
3,YouTube,RGYMRUSSIA,https://www.youtube.com/watch?v=u-oPa_WoPMg,97


(6643, 4)

In [ ]:
df['video_duration'].mean() # средняя продолжительность видеозаписей ~128 сек = 2 минуты 8 сек

np.float64(127.81348788198103)

## Analytics

In [ ]:
platforms = df['platform'].value_counts()

fig1 = px.pie(names=platforms.index,
               values=platforms.values,
               title='Распределение видеохостингов',
               color_discrete_sequence=['deeppink'])

fig1.update_layout(plot_bgcolor='black',
                   paper_bgcolor='black',
                   font_color='white')
fig1.update_layout(height=400, width=500)

fig1.show()

- Подавляющая часть датасета собрана из видеозаписей с платформы YouTube (95.2%), оставшаяся часть (4.77%) - с платформы RuTube. Это обусловлено тем, что платформа видеохостинга "YouTube" гораздо более распространена для размещения видеовыступлений по художественной гимнастике.

In [ ]:
# Подсчет количества видео для каждого канала и их процентного соотношения
channels = df['channel_name'].value_counts()
percentage = df['channel_name'].value_counts(normalize = True)

color_map = {'YouTube': 'darkred', 'RuTube': 'seagreen'}
colors = []
platforms = []

for channel in channels.index:
    platform = df.loc[df['channel_name'] == channel, 'platform'].iloc[0]
    colors.append(color_map[platform])
    platforms.append(platform)

df_plot = pd.DataFrame({
    'channel_name': channels.index,
    'count': channels.values,
    'percentage': percentage.values*100,
    'platform': platforms
})

# Визуализация распределения каналов
fig = px.bar(df_plot,
               x='channel_name',
               y='count',
               title='Распределение каналов',
               labels={'channel_name': 'Канал', 'count': 'Количество'},
               color='platform',  # Используем платформу для задания цвета
               color_discrete_map=color_map,  # цветовая палитра
               text=df_plot['percentage'].round(2).astype(str) + '%')  # Добавляем процент в подписи

fig.update_traces(texttemplate='%{text}', textposition='outside')  # Подписи на столбцах
fig.update_layout(plot_bgcolor='black',  # Цвет фона графика
                  paper_bgcolor='black',  # Цвет фона всей области графика
                  font_color='white')  # Цвет шрифта

fig.show()

- Большая часть (76.08%) видеозаписей датасета  собрана с канала "RGYMRUSSIA" (платформа YouTube). На данном канале содержится больше всего записей видеовыстпулений спортсменов.

In [ ]:
# Средняя длительность видео по каналам и платформам
average_duration = df.groupby(['channel_name', 'platform'])['video_duration'].mean().reset_index()
average_duration.columns = ['channel_name', 'platform', 'average_duration']
color_mapping = {
    'YouTube': 'lime',
    'Rutube': 'yellow'
}
average_duration['color'] = average_duration['platform'].map(color_mapping)

fig = px.bar(
    average_duration,
    x='channel_name',
    y='average_duration',
    color='platform',
    color_discrete_map=color_mapping,
    title='Средняя длительность видео по каналам',
    labels={'channel_name': 'Канал', 'average_duration': 'Средняя длительность (сек.)'},
)

fig.update_layout(
    plot_bgcolor='black',  # Цвет фона графика
    paper_bgcolor='black',  # Цвет фона всей области графика
    font_color='white'  # Цвет шрифта
)
fig.show()

In [ ]:
df.video_duration.mean()  # средняя продолжительность видеозаписей

np.float64(127.81348788198103)

- Средняя продолжительность видеозаписей ~ 128 сек. (= 2 мин. 8 сек.)
- В среднем длительность видео для каждого конкретного канала примерно одинаковая, независимо от платформы (YouTube или RuTube). Различия в длительности между платформами для одного и того же канала невелики.
- Каналы "Noname" и "RGYMRUSSIA" имеют самые длинные видео ( 120-130 секунд), а "Top Tier RG" - самые короткие (около 100 секунд)

In [ ]:
# график распределения длительности видеозаписей в общем
fig = px.histogram(df, x="video_duration", title="Длительность видеозаписей, сек.")

# Обновление макета графика
# fig.update_traces(marker=dict(size=4))  # Настройка размера маркеров
fig.update_layout(plot_bgcolor='black',
                         paper_bgcolor='black',
                         font_color='white')
# Показать график
fig.show()

In [ ]:
# Создание боксплота для длительности видеозаписей
fig = px.box(df,
             y='video_duration',
             title='Длительность видеозаписей, сек.')

fig.update_layout(plot_bgcolor='black',
                  paper_bgcolor='black',
                  font_color='white',
                  yaxis_title='Длительность видео (сек.)')

fig.show()

1. Основная масса видеозаписей имеет продолжительность около 80-99 секунд (1 мин. 20 сек. - 1 мин. 39 сек.), что явлется релевантным, поскольку длительность выступления спортсменки в индивидуальной программе составляет 1.5 минуты. Видеозапись может включать в себя выход спортсменки на площадку, что увеличивает длительность видео.
2. Большой разброс данных: Данные о длительности видеозаписей имеют большой разброс - от 3 сек. до 6000 сек.
3. Наличие выбросов: график показывает наличие незначительного количества видеозаписей с длительностью, значительно превышающей типичную. Эти выбросы достигают 6000 секунд.
4. Длительные видеозаписи могут быть нерелевантными видеозаписями для задачи, например, продолжительными трансляциями, или представлять собой другую категорию видео.

В рамках данный работы анализ проводится для выступлений спортсменов в индивидуальной программе, длительность которой составляет 1 мин. 30 сек. Выступления спорсменов в групповой команде составляет 2 мин. 30 сек. Следовательно, чтобы предотвратить попадание выступления групповых упраженений в выборку, верхний порог установим 130 секунд (2 мин. 10 сек.), тем самым закладывая дополнительно время на возможный выход спортсменки на ковер (если он имеется в видеозаписи).

In [ ]:
# оставляем только вижео длиной <= 130 секунд и >= 90
df = df[(df['video_duration'] <= 130) & (df['video_duration'] >= 90)]
df.shape

(5129, 4)

После проведения фильтрации в итоговый датасет попадант 5129 видео.

# Download videofiles

## Youtube

In [ ]:
# скачивание видеозаписи по ссылке

def download_video_youtube(video_url):
  with yt_dlp.YoutubeDL({
      'extract_audio': False,  # аудио не извлекать
      'format': 'bestvideo',  # в лучшем качестве
      'outtmpl': f'{OUTPUT_PATH}/%(title)s.mp4',  # название видео в формате mp4
      'socket_timeout': 30,  # 30 секунд таймаут
      'retries': 1,       # 1 попытка переподключения
      'skip_download': False,       # скачиваем видео
      'quiet': True  # отключение принтов, варнингов и тд
        }) as video:

          # info_dict = video.extract_info(df_youtube['video_url'].iloc[0])
          # filename = video.prepare_filename(info_dict)  # название файла
          video.download([video_url])  # загружаем видеофайл в папку

# df_youtube['video_url'].progress_apply(lambda video_url: download_video_youtube(video_url))

In [ ]:
# пример, скачать 1 видео по ссылке
download_video_youtube('https://www.youtube.com/watch?v=GE-wcA3twOQ')

In [ ]:
# параллельное скачивание для ускорения процесса
def parallel_download_youtube(params, num_processes=20):
    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        results = list(tqdm(executor.map(download_video_youtube, params)), total = len(params))
    return results

In [ ]:
params = df_youtube['video_url'].values
parallel_download_youtube(params)

0it [00:00, ?it/s]


[download]  97.5% of   31.33MiB at  838.01KiB/s ETA 00:00

ERROR: [youtube] 4W1rc8UIj9I: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


[download]  52.2% of   31.60MiB at    2.12MiB/s ETA 00:07

ERROR: [youtube] -jr5iQ2lZos: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


...

[download]  42.1% of ~  30.54MiB at    1.71MiB/s ETA 00:14 (frag 7/19)

ERROR: [youtube] kWmVJrrUDd0: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


[download]  50.1% of ~   6.19MiB at  929.08KiB/s ETA 00:05 (frag 8/18)

ERROR: [youtube] mX1OCaMgL-E: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


TypeError: list() takes no keyword arguments

In [ ]:
# df_youtube['video_url'].progress_apply(lambda video_url: download_video_youtube(video_url))

## Rutube

In [ ]:
# скачивание видео с рутуба
def download_video_rutube(video_url, output_path = OUTPUT_PATH):
  rt = Rutube(video_url)
  rt.get_best().download(OUTPUT_PATH)   # скачиваем в папку, оставляем название

In [ ]:
for video_url in tqdm(df_rutube['video_url'].values):
  download_video_rutube(video_url)

  0%|          | 0/5 [00:00<?, ?it/s]

2023-02-16 Римарачин Диас Николь ЛЕНТА (Первенство России) (3840x2160) |████████████████████████████████████████| 24/24 [100%] in 18.9s (1.27/s) 


 20%|██        | 1/5 [00:20<01:21, 20.28s/it]

Молодёжная мужская сборная Красноярского края по Художественной гимнастике (3840x2160) |████████████████████████████████████████| 43/43 [100%] in 31.0s (1.39/s) 


 40%|████      | 2/5 [00:52<01:22, 27.38s/it]

...

 60%|██████    | 3/5 [01:17<00:52, 26.28s/it]

2023-02-16 Никольская Софья ЛЕНТА (финал многоборья) (3840x2160) |████████████████████████████████████████| 25/25 [100%] in 17.9s (1.40/s) 


 80%|████████  | 4/5 [01:37<00:23, 23.66s/it]

2023-02-16 Никольская Софья ОБРУЧ (финал многоборья) (3840x2160) |████████████████████████████████████████| 25/25 [100%] in 18.0s (1.39/s) 


100%|██████████| 5/5 [01:57<00:00, 23.47s/it]


# Статистика по собранным элементам

In [ ]:
print(f'Всего скачано {len(os.listdir(OUTPUT_PATH))} видеозаписей')

Всего скачано 1045 видеозаписей


In [ ]:
# сбор статистики
df_elements = pd.DataFrame()
BD_groups, BD_names, BD_counts = [],[],[]
for group in os.listdir(ELEMENTS_PATH):  # равновесия, прыжки, повороты
  df = pd.DataFrame()
  for elem_name in os.listdir(os.path.join(ELEMENTS_PATH,group)):
    BD_groups.append(group) # вращение, поворот, прыжок
    BD_names.append(elem_name)
    BD_counts.append(len(os.listdir(os.path.join(ELEMENTS_PATH,group,elem_name))))# кол-во элементов в папке

df_elements['BD_group'] = BD_groups  # равновесие/прыжок/поворот
df_elements['BD_name'] = BD_names  # название элемента
df_elements['BD_count'] = BD_counts # кол-во видео данного элемента
df_elements['fraction'] = df_elements['BD_count']/df_elements['BD_count'].sum()*100  # доля в датасете
print(f'Всего видеозаписей в датасете: {df_elements.BD_count.sum()}')
df_elements.sort_values('BD_count', ascending = False)

Всего видеозаписей в датасете: 3311


,BD_group,BD_name,BD_count,fraction
15,Повороты,Фуэте: прямая нога в горизонтальном положении,355,10.721836
68,Прыжки,В шпагат с прогибом с поворотом,260,7.852613
10,Повороты,"Задний шпагат без помощи, туловище горизонтально",258,7.792208
23,Повороты,Фуэте боковой шпагат с помощью,164,4.953186
22,Повороты,Циркуль назад,130,3.926306
...,...,...,...,...
43,Равновесия,Задний шпагат с помощью наклон туловища впере...,2,0.060405
38,Равновесия,Арабеск с туловищем вперед в горизонтальном по...,1,0.030202
18,Повороты,В шпагате с помощью с наклоном вперед,1,0.030202
57,Прыжки,«Фуете» с подбивом,1,0.030202


In [ ]:
# топ20
df_elements.sort_values('BD_count', ascending = False).head(20).reset_index(drop = True)

,BD_group,BD_name,BD_count,fraction
0,Повороты,Фуэте: прямая нога в горизонтальном положении,355,10.721836
1,Прыжки,В шпагат с прогибом с поворотом,260,7.852613
2,Повороты,"Задний шпагат без помощи, туловище горизонтально",258,7.792208
3,Повороты,Фуэте боковой шпагат с помощью,164,4.953186
4,Повороты,Циркуль назад,130,3.926306
5,Прыжки,В шпагат с поворотом,130,3.926306
6,Повороты,Фуэте: пассе,116,3.503473
7,Равновесия,Задний шпагат с помощью наклон туловища впере...,106,3.201450
8,Прыжки,Подбивной с прогибом с поворотом,97,2.929629
9,Равновесия,"Боковой шпагат без помощи, туловище в сторону...",97,2.929629


In [ ]:
# меньше всего
df_elements.sort_values('BD_count', ascending = True).reset_index(drop = True).head(40) #.to_excel('Элементы_количество.xlsx', index = False)

,BD_group,BD_name,BD_count,fraction
0,Повороты,В шпагате с помощью с наклоном вперед,1,0.030202
1,Прыжки,«Фуете» с подбивом,1,0.030202
2,Прыжки,«Фуете» с прогибом,1,0.030202
3,Равновесия,Арабеск с туловищем вперед в горизонтальном по...,1,0.030202
4,Равновесия,Задний шпагат с помощью наклон туловища впере...,2,0.060405
5,Повороты,"На животе груди, ноги в положении подбива с по...",2,0.060405
6,Равновесия,Передний шпагат с помощью рук,2,0.060405
7,Прыжки,«Фуете» подбивной с прогибом,2,0.060405
8,Прыжки,Подбивной в кольцо с поворотом,2,0.060405
9,Прыжки,"«Казак» – нога в сторону вверх, вся стопа выше...",3,0.090607


In [ ]:
# всего видеозаписей, медиана
df_elements['BD_count'].sum(), df_elements['BD_count'].median()

(np.int64(3311), 16.5)

In [ ]:
# если не брать элементы, которые встречаются 1 раз
df_elements[df_elements['BD_count'] > 1]['BD_count'].median()

18.5

In [ ]:
# кол-во видео в разрезе для каждой группы БД
grouped_data = df_elements.groupby('BD_group')['BD_count'].count().reset_index()
display(grouped_data)

# круговая диаграмма
fig = px.pie(grouped_data,
             values='BD_count',
             names='BD_group',
             title='Количество классов по группам БД',
             hover_data=['BD_count'])

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(plot_bgcolor='black',
                  paper_bgcolor='black',
                  font_color='white')
fig.show()

,BD_group,BD_count
0,Повороты,24
1,Прыжки,32
2,Равновесия,24


In [ ]:
# кол-во видео в разрезе для каждой группы БД
grouped_data = df_elements.groupby('BD_group')['BD_count'].sum().reset_index()
display(grouped_data)

# круговая диаграмма
fig = px.pie(grouped_data,
             values='BD_count',
             names='BD_group',
             title='Количество видео по группам БД',
             hover_data=['BD_count'])

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(plot_bgcolor='black',
                  paper_bgcolor='black',
                  font_color='white')
fig.show()

,BD_group,BD_count
0,Повороты,1649
1,Прыжки,912
2,Равновесия,750


In [ ]:
# по одному примеру видеозаписи
df_elements[df_elements['BD_count'] == 1]

,BD_group,BD_name,BD_count,fraction
18,Повороты,В шпагате с помощью с наклоном вперед,1,0.030202
38,Равновесия,Арабеск с туловищем вперед в горизонтальном по...,1,0.030202
57,Прыжки,«Фуете» с подбивом,1,0.030202
58,Прыжки,«Фуете» с прогибом,1,0.030202


In [ ]:
fig = px.histogram(df_elements, x="BD_count", title="Количетсво видеозаписей по элементам", nbins=250, color = 'BD_group')
fig.update_layout(plot_bgcolor='black',
                         paper_bgcolor='black',
                         font_color='white')
fig.show()

In [ ]:
fig = px.box(df_elements,
             x='BD_group',
             y='BD_count',
             color='BD_group',
             title='Боксплот количества элементов по группам',
             labels={'BD_count': 'Количество элементов', 'BD_group': 'Группа элемента'},
             hover_data=['BD_name'])
fig.update_layout(plot_bgcolor='black',
                        paper_bgcolor='black',
                        font_color='white')

fig.show()

## duration time

In [ ]:
# средняя длительность выполнения каждого элемента в целом, по прыжкам, поворотам и равновесиям отдельно, самый быстрый и долгий в среднем для каждой группы

In [ ]:
df_videos_markup = pd.DataFrame()
BD_groups, BD_names, name_videos, video_paths = [],[],[],[]

for group in os.listdir(ELEMENTS_PATH):
  for elem_name in os.listdir(os.path.join(ELEMENTS_PATH, group)):
    for video_name in os.listdir(os.path.join(ELEMENTS_PATH, group, elem_name)):
      BD_groups.append(group)
      BD_names.append(elem_name)
      name_videos.append(video_name)
      video_paths.append(os.path.join(ELEMENTS_PATH, group, elem_name, video_name))

In [ ]:
df_videos_markup['BD_group'] = BD_groups
df_videos_markup['BD_name'] = BD_names
df_videos_markup['name_video'] = name_videos
df_videos_markup['video_path'] = video_paths

In [ ]:
durations = []

for vid in tqdm(df_videos_markup['video_path']):
  cap = cv2.VideoCapture(vid)
  if not cap.isOpened():
      print("Error: Could not open video.")

  fps = cap.get(cv2.CAP_PROP_FPS) # частота кадров (кадры в секунду)
  total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT) # кол-во кадров
  if fps == 0:
      durations.append(None)
  else:
      durationInSeconds = round(total_frames / fps, 2)
      durations.append(durationInSeconds)
      # print(f"Продолжительность видео '{vid.split('/')[-1]}': ", durationInSeconds, "сек")

df_videos_markup['durationInSeconds'] = durations

In [ ]:
df_videos_markup.to_csv('video_duration in seconds.csv',
                        index = None
                        )

In [ ]:
df_videos_markup = pd.read_csv('/content/drive/MyDrive/диплом/Datasets/video_duration in seconds (1).csv')
df_videos_markup

,BD_group,BD_name,name_video,video_path,durationInSeconds
0,Повороты,Пассе,поворот пассе — сделано в Clipchamp.mp4,/content/drive/MyDrive/диплом/По элементам/Пов...,1.47
1,Повороты,Пассе,Диана Чугунихина поворт пассе — сделано в Clip...,/content/drive/MyDrive/диплом/По элементам/Пов...,0.70
2,Повороты,Пассе,Диана Чугунихина поворт пассе — сделано в Clip...,/content/drive/MyDrive/диплом/По элементам/Пов...,0.73
3,Повороты,Пассе,Диана Чугунихина поворт пассе — сделано в Clip...,/content/drive/MyDrive/диплом/По элементам/Пов...,0.70
4,Повороты,Пассе,Диана Чугунихина поворт пассе — сделано в Clip...,/content/drive/MyDrive/диплом/По элементам/Пов...,0.97
...,...,...,...,...,...
3306,Прыжки,Подбивной с прогибом со сменой ног,Диана_Чугунихина касаясь прогнувшись со сменои...,/content/drive/MyDrive/диплом/По элементам/Пры...,1.30
3307,Прыжки,Подбивной с прогибом со сменой ног,Диана_Чугунихина касаясь прогнувшись со сменои...,/content/drive/MyDrive/диплом/По элементам/Пры...,1.27
3308,Прыжки,Подбивной с прогибом со сменой ног,краева со сменой касаясь прогнувшись — сделан...,/content/drive/MyDrive/диплом/По элементам/Пры...,1.83
3309,Прыжки,Подбивной с прогибом со сменой ног,Кира_Яблочникова касаясь прогнувшись со сменои...,/content/drive/MyDrive/диплом/По элементам/Пры...,1.60


In [ ]:
agg_df_by_group = df_videos_markup.groupby('BD_group')['durationInSeconds'].agg(['mean', min, max]).reset_index()
agg_df_by_group

<ipython-input-10-076eea08abe0>:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  agg_df_by_group = df_videos_markup.groupby('BD_group')['durationInSeconds'].agg(['mean', min, max]).reset_index()
<ipython-input-10-076eea08abe0>:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  agg_df_by_group = df_videos_markup.groupby('BD_group')['durationInSeconds'].agg(['mean', min, max]).reset_index()


,BD_group,mean,min,max
0,Повороты,1.172421,0.37,5.60
1,Прыжки,1.361393,0.60,3.37
2,Равновесия,1.988773,0.57,5.57


In [ ]:
fig = go.Figure()

# fig.add_trace(go.Bar(
#     x = agg_df_by_group['BD_group'],
#     y = agg_df_by_group['min'],
#     name = "Min",
#     text = round(agg_df_by_group['min'], 2),
#     textposition='outside'
# ))

fig.add_trace(go.Bar(
    x = agg_df_by_group['BD_group'],
    y = agg_df_by_group['mean'],
    name = "Mean",
    text = round(agg_df_by_group['mean'], 2),
    textposition='outside'
))

# fig.add_trace(go.Bar(
#     x = agg_df_by_group['BD_group'],
#     y = agg_df_by_group['max'],
#     name = "Max",
#     text = round(agg_df_by_group['max'], 2),
#     textposition='outside'
# ))

fig.update_layout(title='Сравнение групп элементов по метрикам',
                  xaxis_title='Группы',
                  yaxis_title='Значения, сек',
                  plot_bgcolor='black',
                  paper_bgcolor='black',
                  font_color='white',
                  barmode='group')
fig.show()

In [ ]:
df_duration_by_name = df_videos_markup.groupby(['BD_group', 'BD_name'])['durationInSeconds'].agg(['mean']).reset_index()
df_duration_by_name

,BD_group,BD_name,mean
0,Повороты,Арабеск,1.127059
1,Повороты,Аттитюд,0.969863
2,Повороты,"Боковой шпагат без помощи, туловище в горизон...",1.540370
3,Повороты,Боковой шпагат с помощью,0.917619
4,Повороты,"Боковой шпагат с помощью, туловище в горизонт...",1.363846
...,...,...,...
75,Равновесия,Передний шпагат с помощью рук,1.700000
76,Равновесия,Переход из положения лежа на полу с поднятым т...,1.978387
77,Равновесия,Подъем разгибом,1.468889
78,Равновесия,Свободная нога горизонтально вперед прямая,1.456667


In [ ]:
df_повороты = df_duration_by_name[df_duration_by_name['BD_group'] == 'Повороты']
df_прыжки = df_duration_by_name[df_duration_by_name['BD_group'] == 'Прыжки']
df_равновесия = df_duration_by_name[df_duration_by_name['BD_group'] == 'Равновесия']

In [ ]:
print('Поворот с минимальным средним временем - ', df_повороты[df_повороты['mean'] == df_повороты['mean'].min()]['BD_name'].values[0],
      '=',
      round(df_повороты['mean'].min(), 2),
      'сек'
      )

print('Равновесие с минимальным средним временем - ', df_равновесия[df_равновесия['mean'] == df_равновесия['mean'].min()]['BD_name'].values[0],
      '=',
      round(df_равновесия['mean'].min(), 2),
      'сек'
      )

print('Прыжок с минимальным средним временем - ', df_прыжки[df_прыжки['mean'] == df_прыжки['mean'].min()]['BD_name'].values[0],
      '=',
      round(df_прыжки['mean'].min(), 2),
      'сек'
      )

print()

print('Поворот с максимальным средним временем - ', df_повороты[df_повороты['mean'] == df_повороты['mean'].max()]['BD_name'].values[0],
      '=',
      round(df_повороты['mean'].max(), 2),
      'сек'
      )

print('Равновесие с максимальным средним временем - ', df_равновесия[df_равновесия['mean'] == df_равновесия['mean'].max()]['BD_name'].values[0],
      '=',
      round(df_равновесия['mean'].max(), 2),
      'сек'
      )

print('Прыжок с максимальным средним временем - ', df_прыжки[df_прыжки['mean'] == df_прыжки['mean'].max()]['BD_name'].values[0],
      '=',
      round(df_прыжки['mean'].max(), 2),
      'сек'
      )

Поворот с минимальным средним временем -  Кольцо с помощью с ногой на плече = 0.82 сек
Равновесие с минимальным средним временем -  Пассе = 1.02 сек
Прыжок с минимальным средним временем -  «Фуете» с прогибом = 0.87 сек

Поворот с максимальным средним временем -  На животе груди, ноги в положении подбива с помощью («Ашрам») = 2.46 сек
Равновесие с максимальным средним временем -  Утяшева = 3.83 сек
Прыжок с максимальным средним временем -  Подбивной с поворотом со сменой ног = 2.26 сек


In [ ]:
df_повороты.sort_values(by = 'mean', inplace = True)
df_равновесия.sort_values(by = 'mean', inplace = True)
df_прыжки.sort_values(by = 'mean', inplace = True)

<ipython-input-15-e2c933365c6f>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-15-e2c933365c6f>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-15-e2c933365c6f>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = df_повороты['BD_name'],
    y = df_повороты['mean'],
    text = round(df_повороты['mean'], 2),
    textposition='outside'
))

fig.update_layout(title='Средняя продолжительность поворотов',
                  xaxis_title='Название элемента',
                  yaxis_title='Средняя продолжительность в секундах',
                  plot_bgcolor='black',
                  paper_bgcolor='black',
                  font_color='white',
                  barmode='group')
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = df_равновесия['BD_name'],
    y = df_равновесия['mean'],
    text = round(df_равновесия['mean'], 2),
    textposition='outside'
))

fig.update_layout(title='Средняя продолжительность равновесий в секундах',
                  xaxis_title='Название элемента',
                  yaxis_title='Средняя продолжительность',
                  plot_bgcolor='black',
                  paper_bgcolor='black',
                  font_color='white',
                  barmode='group')
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x = df_прыжки['BD_name'],
    y = df_прыжки['mean'],
    text = round(df_прыжки['mean'], 2),
    textposition='outside'
))

fig.update_layout(title='Средняя продолжительность прыжков',
                  xaxis_title='Название элемента',
                  yaxis_title='Средняя продолжительность в секундах',
                  plot_bgcolor='black',
                  paper_bgcolor='black',
                  font_color='white',
                  barmode='group')
fig.show()